In [2]:
# Imports e configuração
import sqlite3
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

# Caminho absoluto do DB criado pelo script de importação
DB_PATH = Path(r"e:\GitHub-projects\analise-consumo-de-energia\consumo_mensal.db")
print('DB path:', DB_PATH)
assert DB_PATH.exists(), f"Banco de dados não encontrado em {DB_PATH}. Execute o importador primeiro."

DB path: e:\GitHub-projects\analise-consumo-de-energia\consumo_mensal.db


In [3]:
# Lista de tabelas e visualização rápida
with sqlite3.connect(DB_PATH) as conn:
    tables = pd.read_sql_query("SELECT name, type FROM sqlite_master WHERE type IN ('table','view') ORDER BY name", conn)

tables

,name,type
0,ANALISE_CONS_NUMCONS_SAM,table
1,ANALISE_CONS_NUMCONS_SAM_UF,table
2,ANALISE_SETOR_IND_POR_RG,table
3,ANALISE_SETOR_IND_POR_UF,table
4,CONSUMO_BEN_RG_1970_1989,table
5,CONSUMO_ELETROBRAS_1990_2003,table
6,CONSUMO_E_NUMCONS_SAM,table
7,CONSUMO_E_NUMCONS_SAM_UF,table
8,SETOR_INDUSTRIAL_POR_RG,table
9,SETOR_INDUSTRIAL_POR_UF,table


In [4]:
# Funções utilitárias para inspecionar esquema e carregar tabelas
def table_schema(table_name):
    with sqlite3.connect(DB_PATH) as conn:
        return pd.read_sql_query(f"PRAGMA table_info('{table_name}')", conn)

def load_table(table_name, nrows=None):
    with sqlite3.connect(DB_PATH) as conn:
        df = pd.read_sql_query(f"SELECT * FROM '{table_name}'", conn)
    # Tentar converter colunas de data comuns
    for col in ['Data', 'DataExcel', 'DataVersao']:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')
    return df if nrows is None else df.head(nrows)

In [6]:
# Mostrar esquema de algumas tabelas de interesse (se existirem)
candidates = ['CONSUMO_E_NUMCONS_SAM', 'CONSUMO_E_NUMCONS_SAM_UF', 'SETOR_INDUSTRIAL_POR_RG', 'SETOR_INDUSTRIAL_POR_UF']
for t in candidates:
    if t in tables['name'].values:
        print('--- Schema:', t)
        display(table_schema(t))

--- Schema: CONSUMO_E_NUMCONS_SAM


,cid,name,type,notnull,dflt_value,pk
0,0,Data,INTEGER,0,None,0
1,1,DataExcel,TIMESTAMP,0,None,0
2,2,Regiao,TEXT,0,None,0
3,3,Sistema,TEXT,0,None,0
4,4,Classe,TEXT,0,None,0
5,5,TipoConsumidor,TEXT,0,None,0
6,6,Consumo,REAL,0,None,0
7,7,Consumidores,INTEGER,0,None,0
8,8,DataVersao,TIMESTAMP,0,None,0


--- Schema: CONSUMO_E_NUMCONS_SAM_UF


,cid,name,type,notnull,dflt_value,pk
0,0,Data,INTEGER,0,None,0
1,1,DataExcel,TIMESTAMP,0,None,0
2,2,UF,TEXT,0,None,0
3,3,Regiao,TEXT,0,None,0
4,4,Sistema,TEXT,0,None,0
5,5,Classe,TEXT,0,None,0
6,6,TipoConsumidor,TEXT,0,None,0
7,7,Consumo,REAL,0,None,0
8,8,Consumidores,INTEGER,0,None,0
9,9,DataVersao,TIMESTAMP,0,None,0


--- Schema: SETOR_INDUSTRIAL_POR_RG


,cid,name,type,notnull,dflt_value,pk
0,0,Data,INTEGER,0,None,0
1,1,DataExcel,TIMESTAMP,0,None,0
2,2,SetorIndustrial,TEXT,0,None,0
3,3,Regiao,TEXT,0,None,0
4,4,Consumo,REAL,0,None,0
5,5,DataVersao,TIMESTAMP,0,None,0


--- Schema: SETOR_INDUSTRIAL_POR_UF


,cid,name,type,notnull,dflt_value,pk
0,0,Data,INTEGER,0,None,0
1,1,DataExcel,TIMESTAMP,0,None,0
2,2,SetorIndustrial,TEXT,0,None,0
3,3,UF,TEXT,0,None,0
4,4,Regiao,TEXT,0,None,0
5,5,Consumo,REAL,0,None,0
6,6,DataVersao,TIMESTAMP,0,None,0


In [ ]:
# Carregar amostra da tabela principal de consumo (se existir) e inspecionar
main_table = 'CONSUMO_E_NUMCONS_SAM'
if main_table in tables['name'].values:
    df = load_table(main_table)
    print('Shape:', df.shape)
    display(df.head())
    display(df.dtypes)
else:
    print(f"Tabela {main_table} não encontrada no DB. Escolha outra da lista acima.")

In [ ]:
# Exemplo de agregação: consumo por região ao longo do tempo (usa coluna 'DataExcel' tratada)
if main_table in tables['name'].values:
    # Agregar por mês/região
    tmp = df.copy()
    if 'DataExcel' in tmp.columns:
        tmp['mes'] = tmp['DataExcel'].dt.to_period('M').dt.to_timestamp()
    elif 'Data' in tmp.columns:
        # Coluna numérica tipo 20250901 -> tentar parsear
        tmp['mes'] = pd.to_datetime(tmp['Data'].astype(str), errors='coerce').dt.to_period('M').dt.to_timestamp()
    else:
        tmp['mes'] = pd.NaT

    if 'Consumo' in tmp.columns and 'Regiao' in tmp.columns:
        agg = tmp.groupby(['mes','Regiao'], dropna=True)['Consumo'].sum().reset_index()
        pivot = agg.pivot(index='mes', columns='Regiao', values='Consumo')
        display(pivot.tail())
        # Plot
        plt.figure(figsize=(12,6))
        pivot.plot(title='Consumo por Região (mensal)')
        plt.xlabel('Mês')
        plt.ylabel('Consumo')
        plt.tight_layout()
        plt.show()
    else:
        print('Colunas esperadas `Consumo` e `Regiao` não encontradas nesta tabela.')

In [7]:
# Leitura e tratamento de uma tabela analítica (pivot wide) como `ANALISE CONS NUMCONS SAM`
wide_table = 'ANALISE_CONS_NUMCONS_SAM'
if wide_table in tables['name'].values:
    wide = load_table(wide_table)
    print('Shape (wide):', wide.shape)
    display(wide.head())
    # Muitas planilhas de análise exportadas do Excel têm a primeira linha como e a segunda como datas.
    # O usuário pode transformar essa tabela dependendo do formato. Aqui apenas mostramos o conteúdo bruto para inspeção.
else:
    print(f"Tabela pivot {wide_table} não encontrada.")

Shape (wide): (7, 35)


,Soma_de_Consumo,Rótulos_de_Coluna,Unnamed__2,Unnamed__3,Unnamed__4,Unnamed__5,Unnamed__6,Unnamed__7,Unnamed__8,Unnamed__9,...,Unnamed__25,Unnamed__26,Unnamed__27,Unnamed__28,Unnamed__29,Unnamed__30,Unnamed__31,Unnamed__32,Unnamed__33,Unnamed__34
0,Rótulos de Linha,2023-01-01 00:00:00,2023-02-01 00:00:00,2023-03-01 00:00:00,2023-04-01 00:00:00,2023-05-01 00:00:00,2023-06-01 00:00:00,2023-07-01 00:00:00,2023-08-01 00:00:00,2023-09-01 00:00:00,...,2025-01-01 00:00:00,2025-02-01 00:00:00,2025-03-01 00:00:00,2025-04-01 00:00:00,2025-05-01 00:00:00,2025-06-01 00:00:00,2025-07-01 00:00:00,2025-08-01 00:00:00,2025-09-01 00:00:00,Total Geral
1,Centro-Oeste,3239633.556,3235807.195,3443078.312,3509716.187,3376731.672,3366828.748,3270084.243,3546318.432,3730073.289,...,3617074.631,3700286.818,3857464.839,3852413.357,3747947.641,3630921.308,3556944.658,3721645.779,3899225.033,121480004.308
2,Nordeste,7671654.084,7501878.276,7921965.756,7813792.515,7950785.57,7738135.159,7473099.093,7748186.699,8067820.792,...,8477692.486,8210150.863,8362303.753,8451592.68,8709818.041,8278659.042,8161572.147,8279470.761,8334626.832,270513495.455
3,Norte,3179389.379,2962216.743,3327772.25,3213742.839,3393477.977,3383444.196,3426420.95,3601722.962,3679263.311,...,3618409.996,3372379.718,3515606.412,3624334.872,3737569.195,3682194.994,3752463.587,3888141.535,3934685.282,118102362.758
4,Sudeste,20159472.764,20704477.061,22083388.664,21706203.209,20584051.731,20480712.837,20135943.255,20744961.244,21190187.278,...,22558520.961,22876609.153,23753392.306,22355005.027,22059475.302,21242996.835,21317356.192,21495408.125,21709183.563,723252911.22


**Conclusões e próximos passos**
- O notebook lista e inspeciona as tabelas do `consumo_mensal.db`.
- Uma análise inicial de série temporal por `Regiao` é mostrada quando a tabela `CONSUMO_E_NUMCONS_SAM` estiver presente.
- Para análises adicionais posso: completar a importação (o importador teve uma interrupção parcial), padronizar esquemas, e adicionar consultas específicas (por UF, por Classe, por Setor Industrial) e salvar resultados em CSV.

Se quiser, eu executo o importador novamente em modo streaming para garantir que todas as tabelas sejam importadas e então atualizo o notebook com consultas adicionais.